<a href="https://cocl.us/Data_Science_with_Scalla_top"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/SC0103EN/adds/Data_Science_with_Scalla_notebook_top.png" width = 750, align = "center"></a>
 <br/>
<a><img src="https://ibm.box.com/shared/static/ugcqz6ohbvff804xp84y4kqnvvk3bq1g.png" width="200" align="center"></a>"

# Module 2: Preparing Data

## Statistics, Random data and Sampling on Data Frames

### Lesson Objectives 

After completing this lesson, you should be able to: 

- Compute column summary statistics
-	Compute pairwise statistics between series/columns
-	Perform standard sampling on any `DataFrame` 
-	Split any `DataFrame` randomly into subsets
-	Perform stratified sampling onto `DataFrames`
-	Generate Random Data from Uniform and Normal Distributions 


## Summary Statistics for DataFrames 

-	Column summary statistics for DataFrames are available through DataFrame's `describe()` method
-	It returns another `DataFrame`, which contains column-wise results for: 
-	`min`, `max`
-	`mean`, `stddev`
-	`count`
- Column summary statistics can also be computed through DataFrame's `groupBy()` and `agg()` methods, but stddev is not supported
-	It also returns another `DataFrame` with the results

In [ ]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().getOrCreate()
import spark.implicits._

In [ ]:
case class Record(desc: String, value1: Int, value2: Double)

val records = Array(Record("first",1,3.7), Record("second",-2,2.1), Record("third",6,0.7))

val recRDD = sc.parallelize(records)
val recDF = spark.createDataFrame(recRDD)
recDF.show()

val recStats = recDF.describe()
recStats.show()

In [ ]:
// Fetching Results from DataFrame

recStats.filter($"summary" === "stddev").first()
  
val ar1 = recStats.filter($"summary" === "stddev").first().toSeq.drop(2).map(_.toString.toDouble).toArray
println(ar1)
 
val ar2 = recStats.select("value1").map(s => s(0).toString.toDouble).collect()
println(ar2)

In [ ]:
recDF.groupBy().agg(Map("value1" -> "min", "value1" -> "max" ))
 
recDF.groupBy().agg(Map("value1" -> "min", "value2" -> "min"))

In [ ]:
import org.apache.spark.sql.functions._

val recStatsGroup = recDF.groupBy().agg(min("value1"), min("value2"))

recStatsGroup.columns
 
recStatsGroup.first().toSeq.toArray.map(_.toString.toDouble)


## More Statistics on DataFrames 

-	More statistics are available through the stats method in a DataFrame 
-	It returns a `DataFrameStatsFunctions` object, which has the following methods:
-	`corr()` - computes Pearson correlation between two columns
-	`cov()` - computes sample covariance between two columns
- `crosstab()` - Computes a pair-wise frequency table of the given columns 
- `freqItems()` - finds frequent items for columns, possibly with false positives

In [ ]:
val  recDFStat = recDF.stat

println("correlation value1 and value2",recDFStat.corr("value1", "value2"))
println("correlation value1 and value2",recDFStat.cov("value1", "value2"))
recDFStat.freqItems(Seq("value1"), 0.3) .show()

## Sampling on DataFrames 

-	Can be performed on any `DataFrame`
-	Returns a sampled subset of a `DataFrame`
-	Sampling with or without replacement
- Fraction: expected fraction of rows to generate
-	Can be used on bootstrapping procedures

In [ ]:

val df = spark.createDataFrame(Seq((1, 10), (1, 20), (2, 10), (2, 20), (2, 30), (3, 20), (3, 30))).toDF("key", "value")
df.show()
val dfSampled = df.sample(withReplacement=false, fraction=0.3, seed=11L)
dfSampled.show()

## Random Split on DataFrames

-	Can be performed on any DataFrame
-	Returns an array of DataFrames
-	Weighs for the split will be normalized if the do not add up to 1
-	Useful for splitting a data set into training, test and validation sets

In [ ]:
val dfSplit = df.randomSplit(weights=Array(0.3, 0.7), seed=11L) 

dfSplit(0).show()
dfSplit(1).show()

## Stratified Sampling on DataFrames 

-	Can be performed on any `DataFrame` 
- Any column may work as key
-	Without replacement
-	Fraction: specified by key
-	Available as `sampleBy` function in `DataFrameStatFunctions`

In [ ]:
df.stat.
    sampleBy(col="key",
    fractions=Map(1 -> 0.7, 2 -> 0.7, 3 -> 0.7),
    seed=11L).show()

## Random Data Generation 

-	SQL functions to generate columns filled with random values 
-	Two supported distributions: uniform and normal
-	Useful for randomized algorithms, prototyping and performance testing

In [ ]:

import  org.apache.spark.sql.functions.{rand, randn}

val df = spark.range(0, 10)

df.select("id").withColumn("uniform", rand(10L)).
    withColumn("normal", randn(10L)).show()

## Lesson Summary

-	Having completed this lesson, you should be able to:
- Compute column summary statistics 
-	Compute pairwise statistics between series/columns
-	Perform standard sampling on any `DataFrame` 
-	Split any `DataFrame` randomly into subsets 
-	Perform stratified sampling on `DataFrames`
-	Generate Random Data from Uniform and Normal Distributions

### About the Authors

[Petro Verkhogliad](https://www.linkedin.com/in/vpetro) is Consulting Manager at Lightbend. He holds a Masters degree in Computer Science with specialization in Intelligent Systems. He is passionate about functional programming and applications of AI.